In [1]:
# Data handling and numerical computations
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning packages
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Machine learning models (Basic)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Machine learning models (Advanced)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier

# Model evaluation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_recall_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Real-time data and deployment (for future use)
import joblib
import datetime

# Ignoring warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
sns.set(style='whitegrid')

print("Initial packages imported successfully.")

Initial packages imported successfully.


In [2]:


# Select only essential metadata columns from the station file (with correct names)
selected_station_cols = [
    'station_key',
    'suburb',
    'road_functional_hierarchy',
    'road_classification_type',
    'lane_count',
    'wgs84_latitude',
    'wgs84_longitude'
]


# Load station metadata (small file, selective columns)
station_data = pd.read_csv(
    "D:/study/ENGG2112/Major Project/drive-download-20250430T233625Z-001/trafficStations.csv",
    usecols=selected_station_cols
)

# Prepare for chunked loading
chunks = []
chunk_size = 25000  # adjust based on RAM

# Stream traffic data in chunks
for chunk in pd.read_csv(
    "D:/study/ENGG2112/Major Project/drive-download-20250430T233625Z-001/trafficData.csv",
    chunksize=chunk_size,
    low_memory=True
):
    merged_chunk = pd.merge(chunk, station_data, on='station_key', how='left')
    chunks.append(merged_chunk)

# Combine chunks
merged_data = pd.concat(chunks, ignore_index=True)

# Convert core data types after merge to reduce memory
column_types = {
    'station_key': 'int32',
    'traffic_direction_seq': 'int8',
    'cardinal_direction_seq': 'int8',
    'classification_seq': 'int8',
    'year': 'int16',
    'month': 'int8',
    'day': 'int8',
    'day_of_week': 'int8',
    'public_holiday': 'bool',
    'school_holiday': 'bool',
    'daily_total': 'int32'
}
for i in range(24):
    column_types[f'hour_{i:02d}'] = 'int16'

merged_data = merged_data.astype({k: v for k, v in column_types.items() if k in merged_data.columns})

# --- Sanity Check Section ---
print("Columns in merged_data:")
print(merged_data.columns.tolist())

expected_cols = selected_station_cols[1:]  # Skip station_key (already known to exist)

print("\nAre expected station metadata columns present?")
for col in expected_cols:
    print(f"{col}: {'✅' if col in merged_data.columns else '❌'}")

null_check = merged_data[expected_cols].isnull().all(axis=1).sum()
print(f"\nNumber of rows with completely missing station metadata: {null_check}")

print("\nSample merged rows with metadata:")
print(merged_data[['station_key'] + expected_cols].dropna().head())


Columns in merged_data:
['station_key', 'traffic_direction_seq', 'cardinal_direction_seq', 'classification_seq', 'year', 'month', 'day', 'day_of_week', 'public_holiday', 'school_holiday', 'daily_total', 'hour_00', 'hour_01', 'hour_02', 'hour_03', 'hour_04', 'hour_05', 'hour_06', 'hour_07', 'hour_08', 'hour_09', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'road_functional_hierarchy', 'lane_count', 'road_classification_type', 'suburb', 'wgs84_latitude', 'wgs84_longitude']

Are expected station metadata columns present?
suburb: ✅
road_functional_hierarchy: ✅
road_classification_type: ✅
lane_count: ✅
wgs84_latitude: ✅
wgs84_longitude: ✅

Number of rows with completely missing station metadata: 0

Sample merged rows with metadata:
   station_key             suburb road_functional_hierarchy  \
0     15934005  Twelve Mile Creek              Primary Road   
1     15934004  Twelve Mile 

In [ ]:
# Step 1: Sanity check that all required columns exist in the dataset
expected_columns = [
    'station_key', 'traffic_direction_seq', 'cardinal_direction_seq', 'classification_seq',
    'year', 'month', 'day', 'day_of_week', 'public_holiday', 'school_holiday', 'daily_total'
] + [f'hour_{i:02d}' for i in range(24)] + [
    'road_functional_hierarchy', 'lane_count', 'road_classification_type', 
    'suburb', 'wgs84_latitude', 'wgs84_longitude'
]

# Try to list columns in merged_data (if loaded)
try:
    dataset_columns = merged_data.columns.tolist()
    missing_columns = [col for col in expected_columns if col not in dataset_columns]
except Exception:
    missing_columns = expected_columns  # fallback

# Step 2: Redefine component feature groups
hour_cols = [f"hour_{i:02d}" for i in range(24)]
categorical_features = ['road_functional_hierarchy', 'road_classification_type', 'suburb']
numeric_meta = [
    'traffic_direction_seq', 'cardinal_direction_seq', 'classification_seq',
    'year', 'month', 'day', 'day_of_week',
    'public_holiday', 'school_holiday', 'lane_count',
    'wgs84_latitude', 'wgs84_longitude'
]

# Step 3: Combine final feature set
feature_cols = numeric_meta + hour_cols + categorical_features

# Output result
{
    "missing_columns_in_data": missing_columns,
    "feature_cols_count": len(feature_cols),
    "sample_feature_cols": feature_cols[:5] + ["..."] + feature_cols[-5:]
}


In [ ]:
# Step 1: Convert lane_count safely
merged_data['lane_count'] = pd.to_numeric(merged_data['lane_count'], errors='coerce')

# Step 2: Fill missing values intelligently
# You can customize these defaults based on your domain knowledge
merged_data['lane_count'] = merged_data['lane_count'].fillna(2)
merged_data['wgs84_latitude'] = merged_data['wgs84_latitude'].fillna(merged_data['wgs84_latitude'].median())
merged_data['wgs84_longitude'] = merged_data['wgs84_longitude'].fillna(merged_data['wgs84_longitude'].median())

# Step 3: Confirm hour columns and fill any NaNs with 0 (assuming no traffic recorded)
hour_cols = [f"hour_{i:02d}" for i in range(24)]
merged_data[hour_cols] = merged_data[hour_cols].fillna(0)

# Define binary target label before feature validation
merged_data['congested'] = (merged_data['daily_total'] > 1000).astype(int)
merged_data = merged_data.dropna(subset=['road_classification_type'])

# Step 4: Confirm that all required columns are now clean
columns_used = numeric_meta + hour_cols + categorical_features + ['congested']
missing_check = merged_data[columns_used].isnull().sum().sort_values(ascending=False)

# Diagnostic print
print("🔍 Columns with remaining NaNs (should all be 0):")
print(missing_check[missing_check > 0])

# Now proceed safely
X = merged_data[numeric_meta + hour_cols + categorical_features]
y = merged_data['congested']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

print(f"✅ Final shape of training data: {X_train.shape}, test data: {X_test.shape}")


In [ ]:
#Default LightGBM Model
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Train a basic LightGBM model (no balancing, no tuning)
basic_lgb = lgb.LGBMClassifier(random_state=42)
basic_lgb.fit(X_train, y_train)

# Predict on test set
y_pred = basic_lgb.predict(X_test)

# Evaluate
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Report results
report_df = pd.DataFrame({
    "Score": [acc, prec, rec, f1]
}, index=["Accuracy", "Precision", "Recall", "F1 Score"])

print("\n--- Basic LightGBM Classification Report ---")
print(report_df)

# Plot confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Not Congested', 'Congested'],
            yticklabels=['Not Congested', 'Congested'])
plt.title("Basic LightGBM Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()



In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# --- Assume merged_data is already loaded and cleaned, with column 'congested' defined ---

# 1. Group infrequent suburbs into 'Other'
suburb_counts = merged_data['suburb'].value_counts()
common_suburbs = suburb_counts[suburb_counts > 50].index
merged_data['suburb'] = merged_data['suburb'].apply(lambda x: x if x in common_suburbs else 'Other')

# 2. Add peak-hour aggregates
hour_cols = [f"hour_{i:02d}" for i in range(24)]
merged_data['morning_peak'] = merged_data[[f'hour_{i:02d}' for i in range(7, 10)]].sum(axis=1)
merged_data['evening_peak'] = merged_data[[f'hour_{i:02d}' for i in range(16, 19)]].sum(axis=1)

# 3. Define feature columns
categorical_features = ['road_functional_hierarchy', 'road_classification_type', 'suburb']
numeric_meta = ['traffic_direction_seq', 'cardinal_direction_seq', 'classification_seq',
                'year', 'month', 'day', 'day_of_week', 'public_holiday', 'school_holiday',
                'lane_count', 'wgs84_latitude', 'wgs84_longitude', 'morning_peak', 'evening_peak']
feature_cols = numeric_meta + hour_cols + categorical_features

# 4. Remove multicollinear features (threshold 0.95)
corr_matrix = merged_data[feature_cols].corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.95)]
print("Dropping multicollinear features:", to_drop)

feature_cols = [f for f in feature_cols if f not in to_drop]

# 5. Prepare dataset
# One-hot encode categorical variables
X = pd.get_dummies(merged_data[feature_cols], columns=categorical_features, drop_first=True)
y = merged_data['congested']

# 6. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# 7. Apply SMOTE to balance classes on training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Step 1: Initialize LightGBM classifier
lgb_model = lgb.LGBMClassifier(random_state=42)

# Step 2: Train on SMOTE-resampled data
lgb_model.fit(X_train_res, y_train_res)

# Step 3: Predict on test set
y_pred = lgb_model.predict(X_test)

# Step 4: Evaluate
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Step 5: Create performance report DataFrame
report_df = pd.DataFrame({
    "Score": [acc, prec, rec, f1]
}, index=["Accuracy", "Precision", "Recall", "F1 Score"])

# Step 6: Print report and plot confusion matrix
print("\n--- LightGBM Classification Report ---")
print(report_df)

plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Not Congested', 'Congested'],
            yticklabels=['Not Congested', 'Congested'])
plt.title("LightGBM Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()
